In [ ]:
import sys

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from scipy import stats

sys.path.append("./scripts/particles/")

In [ ]:
import data_handler as dh
import metrics
import utils

In [ ]:
outputs = ['PM1', 'PM2.5', 'PM10']
features = [
    'PM1_2.5_OUT', 'PM1_2.5_H_OUT', 
    'PM2.5_OUT', 'PM2.5_H_OUT',
    'PM2.5_10_OUT', 'PM2.5_10_H_OUT',
    'PERSON_NUMBER', 'AIR_PURIFIER', 'WINDOW', 'AIR_CONDITIONER',
    'DOOR', 'WIND_SPEED', 'WIND_DEG', 'HUMIDITY'
]

dates = [
    {"start": "2022-05-07 09:40", "end": "2022-05-17 08:38"},
    {"start": "2022-05-17 11:25", "end": "2022-05-30 23:26"},
    {"start": "2022-06-01 22:40", "end": "2022-07-02 07:00"},
    {"start": "2022-07-02 16:40", "end": "2022-07-09 07:13"},
    {"start": "2022-07-09 14:30", "end": "2022-07-12 10:00"},
    {"start": "2022-07-25 12:00", "end": "2022-08-01 10:00"},
    {"start": "2022-08-03 09:00", "end": "2022-08-11 22:18"},
    {"start": "2022-08-12 12:14", "end": "2022-08-20 00:00"},
    {"start": "2022-08-20 09:38", "end": "2022-09-01 00:00"},
]

In [ ]:
# weather_df = pd.read_csv('../storage/particle/weather.csv', index_col='DATE', parse_dates=True)[['TEMPERATURE', 'WIND_DEG', 'WIND_SPEED', 'HUMIDITY']]
# weather_df['WIND_DEG'] = np.sin(weather_df['WIND_DEG'].values * np.pi / 180)

# df_org = dh.load_data("../storage/particle/data.csv")
# df_org = dh.add_pm_diff(df_org)

# df = pd.concat([df_org, weather_df], axis=1)
# df = df[(df.index >= pd.to_datetime('2022-05-07 09:40')) & (df.index <= pd.to_datetime('2022-09-01 00:00'))]

df = pd.read_csv('../storage/particle/data_for_analysis.csv', index_col='DATE', parse_dates=True)

# 1. Feature correlation
- Pearson correlation(linear)
    - $\rho_{xy}=\frac{\sum_{i=1}^n(x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^n(x_i-\bar{x})^2}\sqrt{\sum_{i=1}^n(y_i-\bar{y})^2}},\;where\,\bar{x}=\frac{1}{n}\sum_{i=1}^nx_i,\,\bar{y}=\frac{1}{n}\sum_{i=1}^ny_i$
- Spearman's rank-based correlation(non-linear)
    - $\sigma_{xy}=1-\frac{6\sum_{i=1}^nd_i^2}{n(n^2-1)},\;where\, d_i:=|rank(x_i)-rank(y_i)|$

In [ ]:
cp_df = df.dropna()
p_res = np.zeros((len(features), len(outputs))) # pearson corr
s_res = np.zeros((len(features), len(outputs))) # spearman corr

for o_idx, output in enumerate(outputs):
    out_data = cp_df[output].values
    for f_idx, feature in enumerate(features):
        feat_data = cp_df[feature].values
        p_corr = np.corrcoef(out_data, feat_data)
        p_res[f_idx, o_idx] = p_corr[0, 1]
        s_corr = stats.spearmanr(out_data, feat_data)
        s_res[f_idx, o_idx] = s_corr.correlation

corr_df = pd.DataFrame(p_res, index=features, columns=outputs)
scorr_df = pd.DataFrame(s_res, index=features, columns=outputs)

In [ ]:
sorted_idc = abs(corr_df['PM2.5']).sort_values(ascending=False).index
ax = corr_df['PM2.5'].loc[sorted_idc].plot(kind='bar', figsize=(18, 10))
ax.set_ylabel('Pearson correlation', fontsize=17)
ax.set_xlabel('Feature', fontsize=17)

In [ ]:
sorted_idc = abs(scorr_df['PM2.5']).sort_values(ascending=False).index
ax = scorr_df['PM2.5'].loc[sorted_idc].plot(kind='bar', figsize=(18, 10))
ax.set_ylabel('Pearson correlation', fontsize=17)
ax.set_xlabel('Feature', fontsize=17)

# 2. Permutation importance
1. Load model
2. Get mse with original data
3. Get mse with permutated data

In [ ]:
import json

outputs = ['PM1', 'PM2.5', 'PM10']
inputs = [
    'PM1_2.5_OUT',
    'PM1_2.5_H_OUT',
    'PM2.5_OUT',
    'PM2.5_H_OUT',
    'PM2.5_10_OUT',
    'PM2.5_10_H_OUT', # 3
    'PERSON_NUMBER',
    'AIR_PURIFIER',
    'WINDOW',
    'AIR_CONDITIONER',
    'DOOR', # 4
    'TEMPERATURE', # 1
    # 'WIND_SPEED', # 2
    'WIND_DEG',
    'HUMIDITY'
]

model_struct = 'gru'
model_num = 3

f = open(f'../projects/particle/model/{model_struct}_{model_num:02d}/config.json', 'r')
config = json.load(f)
f.close()

moving_average_window = config["data"]["moving_average_window"]
moving_average_method = config["data"]["moving_average_method"]
val_size = config["data"]["validation"]
test_size = config["data"]["test"]
train_size = 1 - val_size - test_size
dates = config["data"]["dates"]

in_time_step = config["model"]["window_size"]
out_time_step = 1
offset = config["model"]["offset"]

In [ ]:
weather_df = pd.read_csv('../storage/particle/weather.csv', index_col='DATE', parse_dates=True)[['TEMPERATURE', 'WIND_DEG', 'WIND_SPEED', 'HUMIDITY']]
weather_df['WIND_DEG'] = np.sin(weather_df['WIND_DEG'].values * np.pi / 180)

df_org = dh.load_data("../storage/particle/data.csv")
df_org = dh.add_pm_diff(df_org)

excludes = ['PERSON_NUMBER', 'AIR_PURIFIER', 'AIR_CONDITIONER', 'WINDOW', 'DOOR']
df = dh.apply_moving_average(pd.concat([df_org, weather_df], axis=1), 
                             window=moving_average_window, 
                             method=moving_average_method, 
                             excludes=excludes, 
                             min_periods=1)
df = pd.concat([df, df_org[excludes]], axis=1)
df[excludes] = df[excludes].fillna(method='ffill')
df.dropna(inplace=True)

dfs = dh.trim_df(df, dates)

train_dfs, val_dfs, test_dfs = dh.train_test_split_df(dfs, val_size, test_size)
meta_df = pd.concat(train_dfs).describe()

def to_dataset(_dfs, in_time_step):
    return dh.dfs_to_dataset(_dfs, meta_df, inputs, outputs, in_time_step=in_time_step, out_time_step=1, offset=1, excludes=outputs)

win_size = config["model"]["window_size"]
X_train, y_train = to_dataset(train_dfs, win_size)
X_val, y_val = to_dataset(val_dfs, win_size)
X_test, y_test = to_dataset(test_dfs, win_size)

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(f'../projects/particle/model/{model_struct}_{model_num:02d}/result/model/{model_struct}_{model_num:02d}.h5')

In [ ]:
def get_result(_dfs, output_scaled=False):
    res_dfs = []
    for _df in _dfs:
        df_cp = _df.copy()
        _X, _y = dh.dfs_to_dataset([df_cp], meta_df, inputs, outputs, in_time_step=in_time_step)
        y_hat = model.predict(_X, verbose=False)
        df_cp = df_cp.iloc[in_time_step + out_time_step + offset - 1:]
        for idx, output in enumerate(outputs):
            if output_scaled:
                min_val = meta_df[output]['min']
                max_val = meta_df[output]['max']
                df_cp[output + '_PRED'] = y_hat[:, idx] * (max_val - min_val) + min_val
            else:
                df_cp[output + '_PRED'] = y_hat[:, idx]
        res_dfs.append(df_cp)
    return pd.concat(res_dfs)

In [ ]:
train_res = get_result(train_dfs)
train_res['TYPE'] = 'train'
val_res = get_result(val_dfs)
val_res['TYPE'] = 'val'
test_res = get_result(test_dfs)
test_res['TYPE'] = 'test'

In [ ]:
cols = ["pm1", "pm2.5", "pm10"]
total_res = pd.concat([train_res, val_res, test_res])
res_dfs = [total_res, train_res, val_res, test_res]
res_indices = ["Total", "Train", "Validation", "Test"]
metric_funcs = [metrics.calc_r2, metrics.calc_corrcoef, metrics.calc_nmse, metrics.calc_fb, metrics.calc_b, metrics.calc_a_co, metrics.calc_rmse]
metrics_indices = ["R Square", "Corr", "NMSE", "FB", "B", "a/C", 'rmse']


def calc_metric(_f, _df, _col):
    return _f(_df[_col].values, _df[_col + "_PRED"].values)


for col in cols:
    print(f"======== {col} prediction results ========")
    res_dict = {
        "Metric": metrics_indices,
        "Total": [],
        "Train": [],
        "Validation": [],
        "Test": [],
    }

    for j, m in enumerate(metric_funcs):
        for i, rd in enumerate(res_dfs):
            s = calc_metric(m, rd, col.upper())
            res_dict[res_indices[i]].append(s)

    r_df = pd.DataFrame(res_dict)
    print(r_df)
    print()

In [ ]:
from sklearn.inspection import permutation_importance

mse_data = np.zeros((len(inputs) + 1, len(outputs)))

for o_idx, o in enumerate(outputs):
    mse_data[0, o_idx] = metrics.calc_mse(test_res[o].values, test_res[o+'_PRED'].values)

for f_idx, feat in enumerate(inputs):
    print(f'[INFO] Feature name `{feat}` starts')
    cp_df = df.copy()
    cp_df[feat] = np.random.permutation(cp_df[feat].values)
    p_dfs = dh.trim_df(cp_df, dates)

    train_dfs, val_dfs, test_dfs = dh.train_test_split_df(p_dfs, val_size, test_size)
    X, y = to_dataset(test_dfs, win_size)
    y_hat = model.predict(X)
    for o_idx, o in enumerate(outputs):
        mse_data[f_idx + 1, o_idx] = metrics.calc_mse(y[:, 0, o_idx], y_hat[:, o_idx])

In [ ]:
mse_df = pd.DataFrame(mse_data, index=['Base']+inputs, columns=outputs)

In [ ]:
mse_df.to_csv('fi_gru_without_ws.csv', index_label='FEATURE')

In [ ]:
for feat_idx in mse_df.index:
    for o in outputs:
        mse_df.loc[feat_idx, o + '_DIFF'] = mse_df.loc[feat_idx, o] - mse_df.loc['Base', o]
        mse_df.loc[feat_idx, o + '_INC'] = (mse_df.loc[feat_idx, o + '_DIFF']) / mse_df.loc['Base', o]

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

In [ ]:
def plot_mse(mse_df_in, title):
    sorted_df = mse_df_in[['PM1_INC', 'PM2.5_INC', 'PM10_INC']].copy()
    sorted_df['sum'] = sorted_df.sum(axis=1).values
    sorted_df = sorted_df.sort_values(by='sum', ascending=False)
    sorted_df = sorted_df[['PM1_INC', 'PM2.5_INC', 'PM10_INC']]
    sorted_df = sorted_df * 100
    ax = sorted_df.plot(kind='bar', figsize=(18, 10))
    ax.set_ylabel('Prediction Error(%)', fontsize=17)
    ax.set_xlabel('Feature', fontsize=17)
    ax.set_title(title, fontsize=22)
    ax.set_xticklabels(labels=sorted_df.index, rotation=45)
    ax.legend(outputs)

In [ ]:
plot_mse(mse_df, 'FI without wind speed')

In [ ]:
mse_df = pd.read_csv('fi_with_whole_features.csv', index_col='FEATURE')
plot_mse(mse_df, 'FI with whole features')

In [ ]:
# ax.set_title('Feature Importance without temperature, wind speed, pm2.5~10 hall out, door', fontsize=22)
mse_df = pd.read_csv('fi_without_temp.csv', index_col='FEATURE')
# plot_mse(mse_df, 'FI without T')
plot_mse(mse_df.drop('WINDOW'), 'FI without T (Except window)')

In [ ]:
# ax.set_title('Feature Importance without temperature, wind speed, pm2.5~10 hall out, door', fontsize=22)
mse_df = pd.read_csv('fi_without_temp,ws.csv', index_col='FEATURE')
# plot_mse(mse_df, 'FI without T, WS')
plot_mse(mse_df.drop('WINDOW'), 'FI without T, WS (Except window)')

In [ ]:
# ax.set_title('Feature Importance without temperature, wind speed, pm2.5~10 hall out, door', fontsize=22)
mse_df = pd.read_csv('fi_without_temp,ws,pm25_10_h_out.csv', index_col='FEATURE')
# plot_mse(mse_df, 'FI without T, WS, PM2.5-10 hall out')
plot_mse(mse_df.drop('WINDOW'), 'FI without T, WS, PM2.5-10 hall out (Except window)')

In [ ]:
# ax.set_title('Feature Importance without temperature, wind speed, pm2.5~10 hall out, door', fontsize=22)
mse_df = pd.read_csv('fi_without_temp,ws,pm25_10_h_out,door.csv', index_col='FEATURE')
plot_mse(mse_df, 'FI without T, WS, PM2.5-10 hall out, D')
# plot_mse(mse_df.drop('WINDOW'), 'FI without T, WS, PM2.5-10 hall out, D (Except window)')